### Ray ML Workers


Data is from https://github.com/EliSchwartz/imagenet-sample-images

Simple parallelization of ML workers.

Actor parallelism because stateful.

TODO -- need to make images a git submodule.

In [ ]:
import time
import os

from resnet50 import ResNet50

srn50 = ResNet50()

directory = '../data/imagenet1000'

start_time = time.time()  # Get the current time

# iterate over the sample images
for filename in os.listdir(directory):
    if filename.endswith(".JPEG"):
        try:
            file_path = os.path.join(directory, filename)
            preds = srn50.classify_image(file_path)
            print(f"Filename {filename}: predictions {preds}")
        except:
            print(f"Failed to classify. Probably an image error {filename}.)
            pass
            
end_time = time.time()  # Get the current time again

execution_time = end_time - start_time
print("Execution time: ", execution_time, " seconds")

In [1]:
from rayresnet50 import RayResNet50
import ray
import time
import os

num_actors=4

# script to drive parallel program
ray.init(num_cpus=num_actors, ignore_reinit_error=True)

actors = []
for i in range(num_actors):
    actors.append(RayResNet50.remote())

current_actor = 0

directory = '../data/imagenet1000'
files = os.listdir(directory)
roids = [None] * len(files)

start_time = time.time()  # Get the current time

for i in range(len(files)):
    if files[i].endswith(".JPEG"):
        file_path = os.path.join(directory, files[i])
        roids[i] = (actors[i%num_actors].classify_image.remote(file_path))

for i in range(len(files)):
    try:
        if files[i].endswith(".JPEG"):
            preds = ray.get(roids[i])
            print(f"Filename {files[i]}: predictions {preds}")
    except:
        pass

end_time = time.time()  # Get the current time again

execution_time = end_time - start_time
print("Execution time: ", execution_time, " seconds")


2023-06-21 00:54:59,142	WARNING services.py:1826 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=1.08gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-06-21 00:54:59,269	INFO worker.py:1636 -- Started a local Ray instance.


Filename n01440764_tench.JPEG: predictions ['tench', 'barracouta', 'reel']
Filename n01443537_goldfish.JPEG: predictions ['goldfish', 'tench', 'anemone fish']
Filename n01484850_great_white_shark.JPEG: predictions ['great white shark', 'tiger shark', 'gorilla']
Filename n01491361_tiger_shark.JPEG: predictions ['tiger shark', 'great white shark', 'hammerhead']
Filename n01494475_hammerhead.JPEG: predictions ['hammerhead', 'gar', 'swimming trunks']
Filename n01496331_electric_ray.JPEG: predictions ['electric ray', 'stingray', 'clog']
Filename n01498041_stingray.JPEG: predictions ['electric ray', 'stingray', 'flatworm']
Filename n01514668_cock.JPEG: predictions ['drake', 'cock', 'hen']
Filename n01514859_hen.JPEG: predictions ['hen', 'cock', 'vulture']
Filename n01518878_ostrich.JPEG: predictions ['ostrich', 'traffic light', 'water tower']
Filename n01530575_brambling.JPEG: predictions ['brambling', 'junco', 'coucal']
Filename n01531178_goldfinch.JPEG: predictions ['goldfinch', 'indigo bu

2023-06-21 00:55:35,853	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RayResNet50.classify_image() (pid=66738, ip=172.17.0.2, actor_id=8e43103ee874d2c47ee2cd5e01000000, repr=<rayresnet50.RayResNet50 object at 0xfffeed4286a0>)
  File "/pplectures2022/ebook.2023/examples/rayresnet50.py", line 55, in classify_image
    img_preds = self.model(self.preprocess(imgtensor).unsqueeze(dim=0))
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_presets.py", line 61, in forward
    img = F.normalize(img, mean=self.mean, std=self.std)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py", line 360, in normalize
    return F_t.normalize(tensor, mean=mean, std=std, inplace=inplace)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional

Filename n04371430_swimming_trunks.JPEG: predictions ['swimming trunks', 'sunscreen', 'bikini']
Filename n04376876_syringe.JPEG: predictions ['syringe', 'corkscrew', 'screwdriver']
Filename n04380533_table_lamp.JPEG: predictions ['table lamp', 'lampshade', 'spotlight']
Filename n04389033_tank.JPEG: predictions ['tank', 'amphibian', 'cannon']
Filename n04392985_tape_player.JPEG: predictions ['tape player', 'cassette player', 'CD player']
Filename n04398044_teapot.JPEG: predictions ['teapot', 'coffeepot', 'water jug']
Filename n04399382_teddy.JPEG: predictions ['teddy', 'Band Aid', 'diaper']
Filename n04404412_television.JPEG: predictions ['home theater', 'entertainment center', 'studio couch']
Filename n04409515_tennis_ball.JPEG: predictions ['tennis ball', 'toy terrier', 'Staffordshire bullterrier']
Filename n04417672_thatch.JPEG: predictions ['thatch', 'military uniform', 'marimba']
Filename n04418357_theater_curtain.JPEG: predictions ['theater curtain', 'stage', 'loudspeaker']
Filena

2023-06-21 00:55:36,855	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RayResNet50.classify_image() (pid=66738, ip=172.17.0.2, actor_id=8e43103ee874d2c47ee2cd5e01000000, repr=<rayresnet50.RayResNet50 object at 0xfffeed4286a0>)
  File "/pplectures2022/ebook.2023/examples/rayresnet50.py", line 55, in classify_image
    img_preds = self.model(self.preprocess(imgtensor).unsqueeze(dim=0))
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_presets.py", line 61, in forward
    img = F.normalize(img, mean=self.mean, std=self.std)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py", line 360, in normalize
    return F_t.normalize(tensor, mean=mean, std=std, inplace=inplace)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional

Filename n04465501_tractor.JPEG: predictions ['tractor', 'plow', 'harvester']
Filename n04467665_trailer_truck.JPEG: predictions ['trailer truck', 'moving van', 'tow truck']
Filename n04476259_tray.JPEG: predictions ['tray', 'bathtub', 'tub']
Filename n04479046_trench_coat.JPEG: predictions ['trench coat', 'suit', 'velvet']
Filename n04482393_tricycle.JPEG: predictions ['tricycle', 'jinrikisha', 'bicycle-built-for-two']
Filename n04483307_trimaran.JPEG: predictions ['trimaran', 'catamaran', 'paddle']
Filename n04485082_tripod.JPEG: predictions ['tripod', 'crutch', 'swing']
Filename n04486054_triumphal_arch.JPEG: predictions ['triumphal arch', 'suit', 'Loafer']
Filename n04487081_trolleybus.JPEG: predictions ['trolleybus', 'passenger car', 'streetcar']
Filename n04487394_trombone.JPEG: predictions ['trombone', 'cornet', 'bow tie']
Filename n04493381_tub.JPEG: predictions ['tub', 'bathtub', 'gown']
Filename n04501370_turnstile.JPEG: predictions ['turnstile', 'tripod', 'rule']
Filename n0